In [1]:
##Import required Google Earth Engine python packages and check if they work in python environment
import ee
ee.Initialize()
import geetools
import geemap
import os
from geemap import cartoee
import matplotlib.pyplot as plt
import pandas as pd 

In [2]:
#import the map module that allows for attaching images to an interactive map
Map = geemap.Map()

In [3]:
#Import the river boundary from the Google Earth Engine Server
#Call in river in from a vector file saved into Google Earth Engine
TN_River = ee.FeatureCollection("users/pjf927/TN_River_Main_5000m_Divide_Edit")
#Some function require geometry values to clip features
TN_RiverGeom = TN_River.geometry() 
#Generate a square boundary around the river study area
RiverBounds = TN_RiverGeom.bounds()

#Call in each of the 5000 meter lengths of river
Reach_1 = TN_River.filter(ee.Filter.eq('Reach', 1)).geometry()
Reach_3 = TN_River.filter(ee.Filter.eq('Reach', 2)).geometry() 
Reach_2 = TN_River.filter(ee.Filter.eq('Reach', 3)).geometry() 
Reach_4 = TN_River.filter(ee.Filter.eq('Reach', 4)).geometry() 

#Add river boundary to the map
Map.addLayer(Reach_1, name = 'Reach_1')
Map.addLayer(Reach_2, name = 'Reach_2')
Map.addLayer(Reach_3, name = 'Reach_3')
Map.addLayer(Reach_4, name = 'Reach_4')
#Display the images centered at the whole coverage of the vector file
Map.centerObject(TN_River, 12);
Map

Map(center=[35.145992090816684, -85.15737972252427], controls=(WidgetControl(options=['position', 'transparent…

In [4]:
#Import date csv from cloud score code as a pandas dataframe
df = pd.read_csv(r"D:\School\Adv_Data_Analytics\Project\csv\Cloud_Score_Acquisition_Data.csv", index_col=0)
print(df)

#Convert a dataframe column to a list, check the data type, and print all the dates
df_dates = df['Date'].tolist()
print("df_list is type: ", type(df_dates))
print("Dates in Imagecollection: ", df_dates)

            Date     Sensor  Swath_Row  Cloud_Percent
0     1982-12-07  LANDSAT_4         36       0.036032
1     1983-01-17  LANDSAT_4         36       0.155362
2     1984-03-15  LANDSAT_4         36       0.187627
3     1984-05-19  LANDSAT_5         36       0.004608
4     1984-06-04  LANDSAT_5         36       0.019594
...          ...        ...        ...            ...
1009  2022-10-27  LANDSAT_9         36       0.000656
1010  2022-11-03  LANDSAT_9         36       0.011714
1011  2022-11-04  LANDSAT_8         36       0.021114
1012  2022-11-19  LANDSAT_9         36       0.034967
1013  2022-11-20  LANDSAT_8         36       0.066536

[1014 rows x 4 columns]
df_list is type:  <class 'list'>
Dates in Imagecollection:  ['1982-12-07', '1983-01-17', '1984-03-15', '1984-05-19', '1984-06-04', '1984-06-11', '1984-06-20', '1984-06-27', '1984-09-08', '1984-10-26', '1985-03-26', '1985-04-11', '1985-04-20', '1985-08-10', '1985-10-20', '1985-12-07', '1986-01-08', '1986-01-24', '1986-03-06', 

In [5]:
#Create a function that creates a new dictionary in the image collection called 'Date' and converts the 'system:time_start' list to a "YYYY-MM-dd" format
def addTime(x):
  return x.set('Date', ee.Date(x.get('system:time_start')).format("YYYY-MM-dd")) #Make sure Created date list matches this format

In [6]:
#Call in all landsat imagecollections that have surface reflectance (SR) pre-calcuated 
#Call in Landsat 4 Level 2, Collection 2, Tier 1 dataset 
LS4_SR = (
    ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")
    .filterBounds(TN_River) #Filter only swath grids that cover the TN River Boundary
    .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'],['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']) #Assign imagery bands new names
    .map(addTime) #Apply the mapping function to create the new dictionary list 'Date'
    .filter(ee.Filter.eq('WRS_ROW', 36)) #Filter swath grids that completly cover the largets portion of the TN River Boundary
    .filter(ee.Filter.inList('Date', ee.List(df_dates))) #Filter out dates from the new dictionary list called 'Date' comparing the dates from the csv imported eariler
    .sort('system:time_start') #Sort collection by acquisition time
)

#Call in Landsat 5 Level 2, Collection 2, Tier 1 dataset 
LS5_SR = (
    ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
    .filterBounds(TN_River) #Filter only swath grids that cover the TN River Boundary
    .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'],['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']) #Assign imagery bands new names
    .map(addTime) #Apply the mapping function to create the new dictionary list 'Date'
    .filter(ee.Filter.eq('WRS_ROW', 36)) #Filter swath grids that completly cover the largets portion of the TN River Boundary
    .filter(ee.Filter.inList('Date', ee.List(df_dates))) #Filter out dates from the new dictionary list called 'Date' comparing the dates from the csv imported eariler   
    .sort('system:time_start') #Sort collection by acquisition time
)

#Call in Landsat 7 Level 2, Collection 2, Tier 1 dataset 
LS7_SR = (
    ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
    .filterBounds(TN_River) #Filter only swath grids that cover the TN River Boundary
    .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'],['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']) #Assign imagery bands new names
    .map(addTime) #Apply the mapping function to create the new dictionary list 'Date'
    .filter(ee.Filter.eq('WRS_ROW', 36)) #Filter swath grids that completly cover the largets portion of the TN River Boundary
    .filter(ee.Filter.inList('Date', ee.List(df_dates))) #Filter out dates from the new dictionary list called 'Date' comparing the dates from the csv imported eariler 
    .sort('system:time_start') #Sort collection by acquisition time
)

#Call in Landsat 8 Level 2, Collection 2, Tier 1 dataset 
LS8_SR = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(TN_River) #Filter only swath grids that cover the TN River Boundary
    .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'],['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']) #Assign imagery bands new names
    .map(addTime)
    .filter(ee.Filter.eq('WRS_ROW', 36)) #Filter swath grids that completly cover the largets portion of the TN River Boundary
    .filter(ee.Filter.inList('Date', ee.List(df_dates))) #Filter out dates from the new dictionary list called 'Date' comparing the dates from the csv imported eariler 
    .sort('system:time_start') #Sort collection by acquisition time
)

#Call in Landsat 9 Level 2, Collection 2, Tier 1 dataset 
LS9_SR = (
    ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
    .filterBounds(TN_River) #Filter only swath grids that cover the TN River Boundary
    .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7'],['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']) #Assign imagery bands new names
    .map(addTime) #Apply the mapping function to create the new dictionary list 'Date'
    .filter(ee.Filter.eq('WRS_ROW', 36)) #Filter swath grids that completly cover the largets portion of the TN River Boundary
    .filter(ee.Filter.inList('Date', ee.List(df_dates))) #Filter out dates from the new dictionary list called 'Date' comparing the dates from the csv imported eariler   
    .sort('system:time_start') #Sort collection by acquisition time
)

#Get counts of each of image in an individual sensor's image collection
LS4_count = LS4_SR.size().getInfo()
print("Landsat 4 Images: ", LS4_count)

LS5_count = LS5_SR.size().getInfo()
print("Landsat 5 Images: ", LS5_count)

LS7_count = LS7_SR.size().getInfo()
print("Landsat 7 Images: ", LS7_count)

LS8_count = LS8_SR.size().getInfo()
print("Landsat 8 Images: ", LS8_count)

LS9_count = LS9_SR.size().getInfo()
print("Landsat 9 Images: ", LS9_count)

#Merge all image collections to a newly created image collection called All_SR
All_SR = LS4_SR.merge(LS5_SR.merge(LS7_SR.merge(LS8_SR.merge(LS9_SR)))).sort('system:time_start')

#Get a count of all images filtered in the Landsat Surface Relectance Collection
count = All_SR.size().getInfo()
print("All Cloud Filtered Surface Reflectance Images: ", count)

Landsat 4 Images:  4
Landsat 5 Images:  435
Landsat 7 Images:  398
Landsat 8 Images:  161
Landsat 9 Images:  16
All Cloud Filtered Surface Reflectance Images:  1014


In [ ]:
#Check to see if the sensors are organized by date, by observing back and forth changes when the sensor appears
Sensor = All_SR.aggregate_array("SPACECRAFT_ID").getInfo()
print(Sensor)

In [ ]:
#Clip out all pixels that are IN the TN River boundary and save as a new Image Collection using the lambda function
All_SR_Clip = All_SR.map(lambda image: image.clip(TN_River))

In [ ]:
#Create fucntion that calcualtes the NDTI index 
def NDTI(image):
    blue = image.select('Blue') #Create a variable that selects the blue band
    red = image.select('Red') #Create a variable that selects the red band
    #Run the '(Red - Blue) / (Red + Blue)' equation
    ndti = red.subtract(blue).divide(red.add(blue)).rename('NDTI') #Run the '(Red - Blue) / (Red + Blue)' equation
    return ndti

In [ ]:
#Create fucntion that calcualtes the NSMI index 
def NSMI(image):
    blue = image.select('Blue') #Create a variable that selects the blue band
    green = image.select('Green') #Create a variable that selects the green band
    red = image.select('Red') #Create a variable that selects the red band
    #Run the '((Red + Green) - Blue) / ((Red + Green) + Blue)' equation
    nsmi = red.add(green.subtract(blue)).divide(red.add(green.add(blue))).rename('NSMI') 
    return nsmi

In [ ]:
#Create fucntion that calcualtes the NDSSI index 
def NDSSI(image):
    blue = image.select('Blue') #Create a variable that selects the blue band
    nir = image.select('NIR') #Create a variable that selects the NIR band
    #Run the '(Blue - NIR) / (Blue + NIR)' equation
    ndssi = blue.subtract(nir).divide(blue.add(nir)).rename('NDSSI')
    return ndssi

In [ ]:
#Create fucntion that calcualtes the Quantitative index 
def QUANT(image):
    red = image.select('Red') #Create a variable that selects the red band
    #Run the '2677.2 * (pow(Red, 1.856))' equation
    quant = (red.pow(1.856)).multiply(2677.2).rename('QUANT')
    return quant

In [ ]:
#Apply each equation to the clipped pixels that are in the TN River boundary
NDTI_images = All_SR_Clip.map(NDTI)
NSMI_images = All_SR_Clip.map(NSMI)
NDSSI_images = All_SR_Clip.map(NDSSI)
QUANT_images = All_SR_Clip.map(QUANT)

In [ ]:
#Create a list that collects that orders the images so they can be mapped to the dates
#True_list = collection.toList(collection.size())
NDTI_list = NDTI_images.toList(All_SR_Clip.size())
NSMI_list = NSMI_images.toList(All_SR_Clip.size())
NDSSI_list = NDSSI_images.toList(All_SR_Clip.size())
QUANT_list = QUANT_images.toList(All_SR_Clip.size())

In [ ]:
#NDTI iterate images to add map to layer, export to local directory, and export to Google Drive
for index in range(0, count):
    image = ee.Image(NDTI_list.get(index))
    layer_name = "NDTI_" + str(df_dates[index]) + "_" + str(Sensor[index])
    NDTI_file = os.path.join(r'D:\School\Adv_Data_Analytics\Project\GEE_Output_Rasters\NDTI', layer_name + ".tif")
    geemap.ee_export_image(image, filename = NDTI_file, scale = 30, region = TN_RiverGeom, file_per_band = True)

In [ ]:
#NSMI iterate images to add map to layer, export to local directory, and export to Google Drive
for index in range(0, count):
    image = ee.Image(NSMI_list.get(index))
    layer_name = "NSMI_" + str(df_dates[index]) + "_" + str(Sensor[index])
    NSMI_file = os.path.join(r'D:\School\Adv_Data_Analytics\Project\GEE_Output_Rasters\NSMI', layer_name + ".tif")
    geemap.ee_export_image(image, filename = NSMI_file, scale = 30, region = TN_RiverGeom, file_per_band = True)

In [ ]:
#NDSSI iterate images to add map to layer, export to local directory, and export to Google Drive
for index in range(0, count):
    image = ee.Image(NDSSI_list.get(index))
    layer_name = "NDSSI_" + str(df_dates[index]) + "_" + str(Sensor[index])
    NDSSI_file = os.path.join(r'D:\School\Adv_Data_Analytics\Project\GEE_Output_Rasters\NDSSI', layer_name + ".tif")
    geemap.ee_export_image(image, filename = NDSSI_file, scale = 30, region = TN_RiverGeom, file_per_band = True)

In [ ]:
#QUANT iterate images to add map to layer, export to local directory, and export to Google Drive
#for index in range(0, count):
    image = ee.Image(QUANT_list.get(index))
    layer_name = "QUANT_" + str(df_dates[index]) + "_" + str(Sensor[index])
    #Map.addLayer(image, ndssiVis, layer_name, False)
    QUANT_file = os.path.join(r'D:\School\Adv_Data_Analytics\Project\GEE_Output_Rasters\QUANT', layer_name + ".tif")
    geemap.ee_export_image(image, filename = QUANT_file, scale = 30, region = TN_RiverGeom, file_per_band = True)
    #geemap.ee_export_image_to_drive(image, description = layer_name, folder = 'export', scale = 30) #Takes awhile

In [ ]:
#Clip out all pixels that are AROUND the TN River boundary and save as a new Image Collection using the lambda function
All_SR_Bounds = All_SR.map(lambda image: image.clip(RiverBounds))

In [ ]:
#Applies scaling factors so images are better looking for displaying it graphs
def applyScaleFactors(image):
  opticalBands = image.select(['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']).multiply(0.0000275).add(-0.2)
  return image.addBands(opticalBands, None, True) 

In [ ]:
#Applies the scaling factor to the image collection that has pixels AROUND the TN River boundary
All_SR_Bounds_Scale = All_SR_Bounds.map(applyScaleFactors)

In [ ]:
#Visualization parameters to be added to a graph
vis_params = {
  'bands': ['Red', 'Green', 'Blue'],
  'min': 0.0,
  'max': 0.3,
}

In [ ]:
#Set up Graph Parameters
#The square of the study reagion in WGS84 coorinates
region = [-85.229205793734, 35.098551952466444, -85.1004016243595, 35.216256685353386]

#Size of the figure to be plotted
fig = plt.figure(figsize=(10, 30))

#Add north arrow
north_arrow_dict = {
    "text": "N",
    "xy": (0.10, 0.9),
    "arrow_length": 0.15,
    "text_color": "white",
    "arrow_color": "white",
    "fontsize": 20,
    "width": 5,
    "headwidth": 15,
    "ha": "center",
    "va": "center",
}

#Add Scale Bar
scale_bar_dict = {
    "length": 1,
    "xy": (0.1, 0.05),
    "linewidth": 3,
    "fontsize": 12,
    "color": "white",
    "unit": "mile",
    "ha": "center",
    "va": "bottom",
}

In [ ]:
#Test to see if visualization parameters look good and test out the north arrow and scale bar
#Call in first image of an image collecction
True_Image = ee.Image(All_SR_Bounds_Scale.first())

#Use cartoee to get a map and create one mapping axis
ax = cartoee.get_map(True_Image, region=region, vis_params=vis_params)

#Add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.05, 0.05], linestyle=":")

#Add the north arrow 
cartoee.add_north_arrow(ax, **north_arrow_dict)

#Add the scale bar
cartoee.add_scale_bar_lite(ax, **scale_bar_dict)

#Add a legend that displays the colored bands
red_patch = mpatches.Patch(color='red', label='Red Band')
green_patch = mpatches.Patch(color='green', label='Green Band')
blue_patch = mpatches.Patch(color='blue', label='Blue Band')

#Add the lengend to the side of the map
ax.legend(handles=[red_patch, green_patch, blue_patch], loc='center left', bbox_to_anchor=(1, 0.5))

#Add the title to the top of the map
ax.set_title(label='Above Chickamauga Dam', fontsize=15)

In [ ]:
#Create a timelapse animation of all the images in the image collection with the north arrow and scale bar
cartoee.get_image_collection_gif(
    ee_ic = All_SR_Bounds_Scale, #call in image collection
    out_dir = os.path.expanduser(r"D:\School\Adv_Data_Analytics\Project\Matplot_Figures\True_Color\Time_Lapse_300dpi"), #Set directory
    out_gif = "animation.gif", #navisualization parameters
    region = region, #Set the location of the study area
    fps = 5, #Set oframes per second
    mp4 = True, #Output an mp4 of all png image graphs
    grid_interval = (0.2, 0.2), #Set the grid lines
    plot_title = 'Above Chickamauga Dam', #Add a title
    date_format = 'MM-dd-YYYY', #Set the date of the image graph
    fig_size = (10, 8), #Set font size
    dpi_plot = 300, #Set resolution
    file_format = "png", #Output each graph as a png file
    north_arrow_dict = north_arrow_dict, #Add in north arrow
    scale_bar_dict = scale_bar_dict, #Add in scale bar
    verbose = True, #display each downloaded image graph when it gets created
)